In [ ]:
import pandas as pd
import glob as glob
import numpy as np
from sklearn import linear_model
from sklearn import svm
from sklearn import ensemble
import pipeline_helper

In [ ]:
#columns of interest. 
#Here are the different physio features used in the Machine learning models as regressors

fizz_columns_of_interest = [
    'PPG_Clean', 'PPG_Rate', 'PPG_Peaks',
    'ECG_Rate','ECG_Quality',
    'EDA_Clean', 'EDA_Tonic', 'EDA_Phasic', 'SCR_Onsets', 'SCR_Peaks','SCR_Height', 'SCR_Amplitude',
    'RSP_Amplitude', 'RSP_Rate',
    'corrugator_EMG_Clean',
    'corrugator_EMG_Amplitude', 'corrugator_EMG_Activity',
    'corrugator_EMG_Onsets', 'corrugator_EMG_Offsets',
    'trapezius_EMG_Clean', 'trapezius_EMG_Amplitude',
    'trapezius_EMG_Activity', 'trapezius_EMG_Onsets',
    'trapezius_EMG_Offsets', 'zygomaticus_EMG_Clean',
    'zygomaticus_EMG_Amplitude', 'zygomaticus_EMG_Activity',
    'zygomaticus_EMG_Onsets', 'zygomaticus_EMG_Offsets',
    'RSP_RVT', 'RSP_Phase', 'RSP_Phase_Completion',
    'RSP_Symmetry_PeakTrough', 'RSP_Symmetry_RiseDecay','skt',
    #'sub_num','vid_num',
]

fizz_columns_of_interest = [ 
    'PPG_Rate', 
    'ECG_Rate','ECG_Quality',
    'EDA_Clean', 'EDA_Tonic', 'EDA_Phasic', 'SCR_Onsets', 'SCR_Peaks','SCR_Height', 'SCR_Amplitude',
    'RSP_Amplitude', 'RSP_Rate',
    'corrugator_EMG_Clean',
    'corrugator_EMG_Amplitude', 'corrugator_EMG_Activity',
    'corrugator_EMG_Onsets', 'corrugator_EMG_Offsets',
    'trapezius_EMG_Clean', 'trapezius_EMG_Amplitude',
    'trapezius_EMG_Activity', 'trapezius_EMG_Onsets',
    'trapezius_EMG_Offsets', 'zygomaticus_EMG_Clean',
    'zygomaticus_EMG_Amplitude', 'zygomaticus_EMG_Activity',
    'zygomaticus_EMG_Onsets', 'zygomaticus_EMG_Offsets',
    'RSP_RVT', 'RSP_Phase', 'RSP_Phase_Completion',
    'RSP_Symmetry_PeakTrough', 'RSP_Symmetry_RiseDecay','skt',
    # 'sub_num',
    # 'vid_num',
]

fizz_columns_of_interest_no_emg = [ 
    'PPG_Rate', 
    'ECG_Rate','ECG_Quality',
    'EDA_Clean', 'EDA_Tonic', 'EDA_Phasic', 'SCR_Onsets', 'SCR_Peaks','SCR_Height', 'SCR_Amplitude',
    'RSP_Amplitude', 'RSP_Rate',
    'RSP_RVT', 'RSP_Phase', 'RSP_Phase_Completion',
    'RSP_Symmetry_PeakTrough', 'RSP_Symmetry_RiseDecay','skt',
    # 'sub_num',
    # 'vid_num',
]

fizz_columns_of_interest_no_emg_sub_vid = [ 
    'PPG_Rate', 
    'ECG_Rate','ECG_Quality',
    'EDA_Clean', 'EDA_Tonic', 'EDA_Phasic', 'SCR_Onsets', 'SCR_Peaks','SCR_Height', 'SCR_Amplitude',
    'RSP_Amplitude', 'RSP_Rate',
    'RSP_RVT', 'RSP_Phase', 'RSP_Phase_Completion',
    'RSP_Symmetry_PeakTrough', 'RSP_Symmetry_RiseDecay','skt',
    'sub_num',
    'vid_num',
]

fizz_columns_of_interest_sub_vid = [ 
    'PPG_Rate', 
    'ECG_Rate','ECG_Quality',
    'EDA_Clean', 'EDA_Tonic', 'EDA_Phasic', 'SCR_Onsets', 'SCR_Peaks','SCR_Height', 'SCR_Amplitude',
    'RSP_Amplitude', 'RSP_Rate',
    'corrugator_EMG_Clean',
    'corrugator_EMG_Amplitude', 'corrugator_EMG_Activity',
    'corrugator_EMG_Onsets', 'corrugator_EMG_Offsets',
    'trapezius_EMG_Clean', 'trapezius_EMG_Amplitude',
    'trapezius_EMG_Activity', 'trapezius_EMG_Onsets',
    'trapezius_EMG_Offsets', 'zygomaticus_EMG_Clean',
    'zygomaticus_EMG_Amplitude', 'zygomaticus_EMG_Activity',
    'zygomaticus_EMG_Onsets', 'zygomaticus_EMG_Offsets',
    'RSP_RVT', 'RSP_Phase', 'RSP_Phase_Completion',
    'RSP_Symmetry_PeakTrough', 'RSP_Symmetry_RiseDecay','skt',
    'sub_num',
    'vid_num',
]

zipped_xs = list(zip(['fizz','fizz-emg','fizz-emg_sub_vid','fizz_sub_vid'],[fizz_columns_of_interest,
     fizz_columns_of_interest_no_emg,
     fizz_columns_of_interest_no_emg_sub_vid,
     fizz_columns_of_interest_sub_vid]))

In [ ]:
#this code window is a copy of "pipeline_helper.py" with minor edits.
#Check pipeline helper for additional information.

import glob,os
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
from math import sqrt
from sklearn.metrics import mean_squared_error, r2_score
from scipy.signal import resample


def get_file_paths(subjects, video_numbers, x_folder, y_folder, file_type = '1hz_zscored'):
    x_files = []
    y_files = []

    for subject in subjects:
        for video in video_numbers:
            x_pattern = os.path.join(x_folder,
                                     f"sub_{int(subject)}_vid_{int(video)}_resampled_{file_type}.csv")
#             y_pattern = os.path.join(y_folder, f"sub_{int(subject)}_vid_{int(video)}.csv")
            y_pattern = os.path.join(y_folder, f"sub_{int(subject)}_vid_{int(video)}_resampled_{file_type}.csv")
            #print(x_pattern)

            x_files.extend(glob.glob(x_pattern))
            y_files.extend(glob.glob(y_pattern))

    return x_files, y_files

#cross validation for scenario 1
def k_fold_cross_validation_time(subjects, video_numbers, x_folder, y_folder, file_type = '1hz'):
    train_x, train_y = get_file_paths(subjects, video_numbers, x_folder+'train/', y_folder+'train/', file_type)
    test_x, test_y = get_file_paths(subjects, video_numbers, x_folder+'test/', y_folder+'test/', file_type)

 
    fold_data = [((train_x, train_y), (test_x, test_y))]
    return fold_data

#cross validation for scenario 2
def k_fold_cross_validation(subjects, video_numbers, k, x_folder, y_folder, file_type = '1hz_zscored'):
    subject_indices = np.arange(len(subjects))
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    
    fold_data = []
    for train_index, test_index in kf.split(subject_indices):
        train_subjects = subjects[train_index]
        test_subjects = subjects[test_index]

        train_x, train_y = get_file_paths(train_subjects, video_numbers, x_folder, y_folder, file_type=file_type)
        test_x, test_y = get_file_paths(test_subjects, video_numbers, x_folder, y_folder,  file_type=file_type)

        fold_data.append(((train_x, train_y), (test_x, test_y)))

    return fold_data

#cross validation for scenario 3 and 4
def k_fold_cross_validation_video(subjects, video_numbers, k, x_folder, y_folder, file_type = '1hz_zscored'):
    video_indices = np.arange(len(video_numbers))
    kf = KFold(n_splits=k, shuffle=True, random_state=42)

    fold_data = []
    for train_index, test_index in kf.split(video_indices):
        train_videos = video_numbers[train_index]
        test_videos = video_numbers[test_index]

        train_x, train_y = get_file_paths(subjects, train_videos, x_folder, y_folder, file_type=file_type)
        test_x, test_y = get_file_paths(subjects, test_videos, x_folder, y_folder, file_type=file_type)

        fold_data.append(((train_x, train_y), (test_x, test_y)))

    return fold_data

import torch
from torch.utils.data import Dataset

class EmognitionDataset(Dataset):
    def __init__(self, x_files, y_files, x_columns=None, y_columns=None, downsampling_factor_x=1,downsampling_factor_y=1, upsample_factor_x=1, upsample_factor_y=1):
        self.x_data = []
        self.y_data = []  
        self.sub_num_data = []
        self.vid_num_data = []

        for file in x_files:
            x_df = pd.read_csv(file)
            x_array = x_df[x_columns].values[::downsampling_factor_x]
            if x_array.shape[0] == 1:
                x_array = np.nan_to_num(x_array)
            
            if upsample_factor_x > 1:
                x_array = self.upsample(x_array, upsample_factor_x)
            
            self.x_data.append(x_array)

            sub_num_array = x_df['sub_num'].values[::downsampling_factor_x]
            vid_num_array = x_df['vid_num'].values[::downsampling_factor_x]
            
            if upsample_factor_x > 1:
                new_num_samples = sub_num_array.shape[0] * upsample_factor_x
                sub_num_array = resample(sub_num_array, new_num_samples)
                vid_num_array = resample(vid_num_array, new_num_samples)
            
            self.sub_num_data.append(sub_num_array)
            self.vid_num_data.append(vid_num_array)
     

        for file in y_files:
            y_df = pd.read_csv(file, usecols=y_columns)
            y_array = y_df.values[::downsampling_factor_y]
            
            if upsample_factor_y > 1:
                new_num_samples = y_array.shape[0] * upsample_factor_y
                if y_array.shape[0] == 1:
                     y_array = resample(y_array, new_num_samples)
                else:
                    y_array = resample(y_array.squeeze(), new_num_samples)
            
            self.y_data.append(y_array)
            
        for i in range(len(self.x_data)):
            # make sure they are the same length between x_file and y_files
#             if upsample_factor_y > 1:
#                 print(len(self.x_data[i]), len(self.y_data[i]))
            min_len = min(len(self.x_data[i]), len(self.y_data[i]))
            self.x_data[i] = self.x_data[i][:min_len]
            self.y_data[i] = self.y_data[i][:min_len]
            self.sub_num_data[i] = self.sub_num_data[i][:min_len]
            self.vid_num_data[i] = self.vid_num_data[i][:min_len]

            
            # Combine x and y data, drop rows with NaN values
            combined_df = pd.concat([pd.DataFrame(self.x_data[i], columns=x_columns),
                         pd.DataFrame(self.y_data[i], columns=y_columns),
                         pd.DataFrame(self.sub_num_data[i], columns=['sub_num_dataset']),
                         pd.DataFrame(self.vid_num_data[i], columns=['vid_num_dataset'])], axis=1)

            # Drop rows with NaN values
            combined_df = combined_df.dropna()

            # Extract cleaned x, y, sub_num, and vid_num data
            self.x_data[i] = combined_df[x_columns].values
            self.y_data[i] = combined_df[y_columns].values
            self.sub_num_data[i] = combined_df['sub_num_dataset'].values
            self.vid_num_data[i] = combined_df['vid_num_dataset'].values
  
    def upsample(self, x_array, upsample_factor_x):
        if x_array.shape[0] > 1:
            new_num_samples = x_array.shape[0] * upsample_factor_x
            col_mean = np.nanmean(x_array, axis=0)
            inds = np.where(np.isnan(x_array))
            x_array[inds] = np.take(col_mean, inds[1])
            x_array = resample(x_array.squeeze(), new_num_samples)
        elif x_array.shape[0] == 1 or x_array.ndim == 1:
            new_num_samples = x_array.shape[0] * upsample_factor_x
            col_mean = np.nanmean(x_array, axis=0)
            inds = np.where(np.isnan(x_array))
            x_array[inds] = np.take(col_mean, inds[1])
            x_array = resample(x_array, new_num_samples)
        return x_array
            
    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, index):
        x = self.x_data[index]
        y = self.y_data[index]
        sub_num = self.sub_num_data[index]
        vid_num = self.vid_num_data[index]

        return x, y, sub_num, vid_num


from sklearn.metrics import mean_squared_error, r2_score
import joblib
from math import sqrt
from scipy.signal import resample

#upsample the predictions from X hz to Y hz
def upsample_predictions(predictions, factor):
    """
    Upsample the predictions by a given factor using resample function.
    
    :param predictions: np.array of predictions
    :param factor: int, factor to upsample the predictions
    :return: np.array of upsampled predictions
    """
    new_num_samples = predictions.shape[0] * factor
    upsampled_predictions = resample(predictions.squeeze(), new_num_samples)
    
    return upsampled_predictions.reshape(-1, 1)

def TrainModel(model, fold_data, x_columns, y_columns, save_file_name, 
                        downsampling_factor_y=1, 
                        downsampling_factor_x=1,verbose = True):
    '''
    which_features: x column names
    which_y: y column name
    save_file_name: descriptive name for the specific version of this model.
    e.g., LinearReg_arousal_
    
    '''
    test_res = []
    all_pred_y = []
    all_sub_num = []
    all_vid_num = []
    for f, fold in enumerate(fold_data):
        train_x_files, train_y_files = fold[0]
        test_x_files, test_y_files = fold[1]

        train_dataset = EmognitionDataset(
            train_x_files, train_y_files, x_columns=x_columns, y_columns=y_columns,
            downsampling_factor_y=downsampling_factor_y,
            downsampling_factor_x=downsampling_factor_x
        )
        
        test_dataset = EmognitionDataset(
            test_x_files, test_y_files, x_columns=x_columns, y_columns=y_columns,
            downsampling_factor_y=downsampling_factor_y,
            downsampling_factor_x=downsampling_factor_x, upsample_factor_x=20, upsample_factor_y=20
        )
    

        train_x = np.vstack([train_dataset[i][0] for i in range(len(train_dataset))])
        train_y = np.vstack([train_dataset[i][1] for i in range(len(train_dataset))])

        test_x = np.vstack([test_dataset[i][0] for i in range(len(test_dataset))])
        test_y = np.vstack([test_dataset[i][1] for i in range(len(test_dataset))])
        #print([test_dataset[i][2] for i in range(len(test_dataset))])
        test_sub = np.hstack([test_dataset[i][2] for i in range(len(test_dataset))])
        test_vid = np.hstack([test_dataset[i][3] for i in range(len(test_dataset))])

        
        model.fit(train_x, train_y)

       # Make predictions
        pred_y = model.predict(test_x).reshape(-1, 1)
        

        # Calculate metrics with upsampled predictions
        rmse = sqrt(mean_squared_error(test_y, pred_y))
        test_res.append(rmse)
        r2 = r2_score(test_y, pred_y)
        
        
        if verbose:
            print(f'---------------------- fold {f+1} ---------------------------')
            print(f'rmse = {rmse:.3f}')
            print(f'r2 score = {r2:.3f}')

        # save: 
        all_pred_y.append(pred_y)
        all_sub_num.append(test_sub)
        all_vid_num.append(test_vid)

  #  return all_pred_y
    all_pred_y_concat = np.vstack(all_pred_y)
    all_sub_num_concat = np.hstack(all_sub_num)
    all_vid_num_concat = np.hstack(all_vid_num)

    all_pred_y_df = pd.DataFrame({'sub': all_sub_num_concat, 'vid': all_vid_num_concat, 'pred_y': all_pred_y_concat[:,0]})
    all_pred_y_df.to_csv(save_file_name + '.csv', index=False)


    joblib.dump(model, save_file_name + '.joblib')
    return test_res, all_pred_y_df, model

def train_model_no_eval(model, 
                        train_x_files,
                        train_y_files,
                        x_columns, 
                        y_columns, 
                        save_file_name, 
                        downsampling_factor_y=20, 
                        downsampling_factor_x=1, 
                        verbose = True):
    '''
    which_features: x column names
    which_y: y column name
    save_file_name: descriptive name for the specific version of this model.
    e.g., LinearReg_arousal_
    
    '''
    train_x_files, train_y_files

    train_dataset = EmognitionDataset(
        train_x_files, 
        train_y_files,
        x_columns=x_columns,
        y_columns=y_columns,
        downsampling_factor_y=downsampling_factor_y,
        downsampling_factor_x=downsampling_factor_x
    )

    train_x = np.vstack([train_dataset[i][0] for i in range(len(train_dataset))])
    train_y = np.vstack([train_dataset[i][1] for i in range(len(train_dataset))])
    model.fit(train_x, train_y)
    joblib.dump(model, save_file_name + '.joblib')
    return model


This code is designed to perform model training and evaluation for each scenario.

For instance, for scenario 2:

1. It initializes a list of models (`models`) and their corresponding names (`model_names`). It uses Elastic Net, Random Forest, and Support Vector Regression with Radial basis function (RBF) kernel.

2. It sets up a list of target variables (`y_columns_list`) for arousal and valence.

3. It specifies the scenario number as 2 and iterates over the target variables in `y_columns_list`.

4. It iterates over the file type (in this case, '1hz') and the models in the `models` list along with their names in the `model_names` list.

5. Inside the loop, it iterates over the folds (0 to 4) for Scenario 2 and loads the subject numbers and video numbers for each fold.

6. It sets up the file paths for the input physiological data (`x_folder`) and the annotation data (`y_folder`).

7. It performs k-fold cross-validation with the given input data and parameters. The `TrainModel` function is called to train the model using the specified input features and target variables.

8. It calculates the Root Mean Squared Error (RMSE) for each fold and stores the results in a list (`rmses`).

9. It saves the results to a CSV file, which includes the fold number, input feature type, and the RMSE for each fold.

In summary, this code trains and evaluates several models (Elastic Net, Random Forest, and Support Vector Regression with RBF kernel) using k-fold cross-validation on the input physiological data and target variables (arousal and valence) for Scenario 2 of the Emognition 2023 challenge. The performance results are saved to CSV files for further analysis.

Each scenario works in a similar way in terms of the code structure. However, scenarios differs in calling different cross validation functions, different features, and different machine learning models.

In [ ]:
#SCENARIO2 

y_columns_list = [['arousal'],['valence']]
#y_columns_list = [['valence']]
models = []
model_names = []
# specify the model
models.append(linear_model.ElasticNet())
model_names.append('Elastic_Net')

models.append(ensemble.RandomForestRegressor())
model_names.append('random_forest_default')

models.append(svm.SVR(kernel='rbf'))
model_names.append('SVR_RBF')



# specify the data info
scenario = 2

# where to save the results:
# TO DO: add which Features in the save_file_name to test features....
for y_columns in y_columns_list:
    for file_type in ['1hz']:
        for model_name, model in list(zip(model_names,models)):
            print(model_name)
            fold_arr,x_names, rmses =[],[],[]
            for fold_num in range(0,5):
                print('folder ', fold_num)
                for x_name,x_cols in zipped_xs: 
                    save_file_name = f'/work/abslab/emognition_2023_challenge/results/results_30s/scenario_{scenario}/fold_{fold_num}/{model_name}_{y_columns[0]}_{x_name}'

                    subjects = np.loadtxt(f'/work/abslab/emognition_2023_challenge/tests/scenario_{scenario}_fold_{fold_num}_sub_list.txt')  # Subject numbers
                    video_numbers = np.loadtxt(f'/work/abslab/emognition_2023_challenge/tests/scenario_{scenario}_fold_{fold_num}_video_list.txt')  # Video numbers
                    k = 5  # Number of folds

                    x_folder = f"/work/abslab/emognition_2023_challenge/file_prep_30sec/scenario_{scenario}/fold_{fold_num}/train/physiology"
                    y_folder = f"/work/abslab/emognition_2023_challenge/file_prep_30sec/scenario_{scenario}/fold_{fold_num}/train/annotations"

                    # x_files, y_files = get_file_paths(subjects, video_numbers, x_folder, y_folder)
                    fold_data = k_fold_cross_validation(subjects,
                                                                              video_numbers,
                                                                              k, 
                                                                              x_folder,
                                                                              y_folder, 
                                                                              file_type)
            
                    rmse_list,_,_ = TrainModel(model,fold_data,x_cols,y_columns,save_file_name,
                                                                              verbose=False)
                    fold_arr.append(fold_num),x_names.append(x_name),rmses.append(np.mean(rmse_list))

            pd.DataFrame({'folds':fold_arr,'x_col_type':x_names,
                          'rmses':rmses}).to_csv(
                f'/work/abslab/emognition_2023_challenge/results/results_30s/scenario_{scenario}_{model_name}_across_folds_{y_columns[0]}_{file_type}.csv')      

In [ ]:
#SCENARIO3

y_columns_list = [['arousal'],['valence']]
#y_columns_list = [['valence']]
models = []
model_names = []
# specify the model
models.append(linear_model.ElasticNet())
model_names.append('Elastic_Net')

models.append(ensemble.RandomForestRegressor())
model_names.append('random_forest_default')

models.append(svm.SVR(kernel='rbf'))
model_names.append('SVR_RBF')

# model = linear_model.LinearRegression()
# model_name = 'LinearReg'


# specify the data info
scenario = 3

# where to save the results:
# TO DO: add which Features in the save_file_name to test features....
for y_columns in y_columns_list:
    for file_type in ['1hz']:
        for model_name, model in list(zip(model_names,models)):
            print(model_name)
            fold_arr,x_names, rmses =[],[],[]
            for fold_num in range(0,4):
                print('folder ', fold_num)
                for x_name,x_cols in zipped_xs: 
                    save_file_name = f'/work/abslab/emognition_2023_challenge/results/results_30s/scenario_{scenario}/fold_{fold_num}/{model_name}_{y_columns[0]}_{x_name}'

                    subjects = np.loadtxt(f'/work/abslab/emognition_2023_challenge/tests/scenario_{scenario}_fold_{fold_num}_sub_list.txt')  # Subject numbers
                    video_numbers = np.loadtxt(f'/work/abslab/emognition_2023_challenge/tests/scenario_{scenario}_fold_{fold_num}_video_list.txt')  # Video numbers
                    k = 4  # Number of folds

                    x_folder = f"/work/abslab/emognition_2023_challenge/file_prep_30sec/scenario_{scenario}/fold_{fold_num}/train/physiology"
                    y_folder = f"/work/abslab/emognition_2023_challenge/file_prep_30sec/scenario_{scenario}/fold_{fold_num}/train/annotations"

                    # x_files, y_files = get_file_paths(subjects, video_numbers, x_folder, y_folder)
                    fold_data = k_fold_cross_validation_video(subjects,
                                                                              video_numbers,
                                                                              k, 
                                                                              x_folder,
                                                                              y_folder, 
                                                                              file_type)
            
                    rmse_list,_,_ = TrainModel(model,fold_data,x_cols,y_columns,save_file_name,
                                                                              verbose=False)
                    fold_arr.append(fold_num),x_names.append(x_name),rmses.append(np.mean(rmse_list))

            pd.DataFrame({'folds':fold_arr,'x_col_type':x_names,
                          'rmses':rmses}).to_csv(
                f'/work/abslab/emognition_2023_challenge/results/results_30s/scenario_{scenario}_{model_name}_across_folds_{y_columns[0]}_{file_type}.csv')      

In [ ]:
#SCENARIO4

y_columns_list = [['arousal'],['valence']]
#y_columns_list = [['valence']]
models = []
model_names = []
# specify the model
models.append(linear_model.ElasticNet())
model_names.append('Elastic_Net')

models.append(ensemble.RandomForestRegressor())
model_names.append('random_forest_default')

models.append(svm.SVR(kernel='rbf'))
model_names.append('SVR_RBF')

# model = linear_model.LinearRegression()
# model_name = 'LinearReg'


# specify the data info
scenario = 4

# where to save the results:
# TO DO: add which Features in the save_file_name to test features....
for y_columns in y_columns_list:
    for file_type in ['1hz']:
        for model_name, model in list(zip(model_names,models)):
            print(model_name)
            fold_arr,x_names, rmses =[],[],[]
            for fold_num in range(0,2):
                print('folder ', fold_num)
                for x_name,x_cols in zipped_xs: 
                    save_file_name = f'/work/abslab/emognition_2023_challenge/results/results_30s/scenario_{scenario}/fold_{fold_num}/{model_name}_{y_columns[0]}_{x_name}'

                    subjects = np.loadtxt(f'/work/abslab/emognition_2023_challenge/tests/scenario_{scenario}_fold_{fold_num}_sub_list.txt')  # Subject numbers
                    video_numbers = np.loadtxt(f'/work/abslab/emognition_2023_challenge/tests/scenario_{scenario}_fold_{fold_num}_video_list.txt')  # Video numbers
                    k = 2  # Number of folds

                    x_folder = f"/work/abslab/emognition_2023_challenge/file_prep_30sec/scenario_{scenario}/fold_{fold_num}/train/physiology"
                    y_folder = f"/work/abslab/emognition_2023_challenge/file_prep_30sec/scenario_{scenario}/fold_{fold_num}/train/annotations"

                    # x_files, y_files = get_file_paths(subjects, video_numbers, x_folder, y_folder)
                    fold_data = k_fold_cross_validation_video(subjects,
                                                                              video_numbers,
                                                                              k, 
                                                                              x_folder,
                                                                              y_folder, 
                                                                              file_type)
            
                    rmse_list,_,_ = TrainModel(model,fold_data,x_cols,y_columns,save_file_name,
                                                                              verbose=False)
                    fold_arr.append(fold_num),x_names.append(x_name),rmses.append(np.mean(rmse_list))

            pd.DataFrame({'folds':fold_arr,'x_col_type':x_names,
                          'rmses':rmses}).to_csv(
                f'/work/abslab/emognition_2023_challenge/results/results_30s/scenario_{scenario}_{model_name}_across_folds_{y_columns[0]}_{file_type}.csv')      

In [ ]:
# #PRE-SCENARIO 1
# import os
# import numpy as np
# import glob
# import pandas as pd
# from sklearn.model_selection import train_test_split

# # creating cv data for scenario 1 for 1hz
# # this do not resample the y files
# x_data_path = f"/work/abslab/emognition_2023_challenge/file_prep_30sec/scenario_1/train/physiology/"
# y_data_path = f"/work/abslab/emognition_2023_challenge/data/scenario_1/train/annotations/"
# s_data_path = f"/work/abslab/emognition_2023_challenge/file_prep_30sec/scenario_1/train/"


# video_numbers = np.loadtxt("/work/abslab/emognition_2023_challenge/data/scenario_1/train/video_list.txt")
# subjects = np.loadtxt("/work/abslab/emognition_2023_challenge/data/scenario_1/train/sub_list.txt")

# file_pattern = '1hz'
# for v in video_numbers:  
#     print(int(v))
  
#     all_files = glob.glob(f'/work/abslab/emognition_2023_challenge/data/scenario_1/train/annotations/*vid_{int(v)}.csv')
#     this_df = pd.read_csv(all_files[1])
#     video_length = len(this_df['time'])

#     test_size = 601/(video_length + 601)

#     x_files, y_files = get_file_paths(subjects, [v], x_data_path, y_data_path, file_pattern)
    
#     for i in range(len(x_files)):

#         X = pd.read_csv(x_files[i])
#         y = pd.read_csv(y_files[i])
#         y_ = pd.DataFrame()
        
#         for i in range(len(X)):
#             t = X.iloc[i,0]
#             newy = y.loc[y['time'] == t]
#             y_ = y_.append(newy)

#         file = y_files[i]
#         vid_id = file.split('vid_')[1].split('.csv')[0]

#         sub_id = file.split('sub_')[1].split('_vid')[0]
        
#         train_x, test_x, train_y, test_y = train_test_split(X, y_, test_size=test_size, shuffle=False)
        
#         train_x.to_csv(s_data_path + f'cv/train/sub_{sub_id}_vid_{vid_id}_resampled_{file_pattern}')
#         test_x.to_csv(s_data_path + f'cv/test/sub_{sub_id}_vid_{vid_id}_resampled_{file_pattern}')



# #SCENARIO 1


# y_columns_list = [['arousal'],['valence']]
# #y_columns_list = [['valence']]
# models = []
# model_names = []
# # specify the model
# models.append(linear_model.ElasticNet())
# model_names.append('Elastic_Net')

# models.append(ensemble.RandomForestRegressor())
# model_names.append('random_forest_default')

# models.append(svm.SVR(kernel='rbf'))
# model_names.append('SVR_RBF')

# # model = linear_model.LinearRegression()
# # model_name = 'LinearReg'


# # specify the data info
# scenario = 1

# # where to save the results:
# # TO DO: add which Features in the save_file_name to test features....
# for y_columns in y_columns_list:
#     for file_type in ['1hz']:
#         for model_name, model in list(zip(model_names,models)):
#             print(model_name)
#             x_names, rmses =[],[]
#             for x_name,x_cols in zipped_xs: 
#                 save_file_name = f'/work/abslab/emognition_2023_challenge/results/results_30s/scenario_{scenario}/{model_name}_{y_columns[0]}_{x_name}'

#                 subjects = np.loadtxt(f'/work/abslab/emognition_2023_challenge/data/scenario_{scenario}/train/sub_list.txt')  # Subject numbers
#                 video_numbers = np.loadtxt(f'/work/abslab/emognition_2023_challenge/data/scenario_{scenario}/train/video_list.txt')  # Video numbers
#                 #k = 1  # Number of folds

#                 x_folder = f"/work/abslab/emognition_2023_challenge/file_prep_30sec/scenario_{scenario}/train/physiology"
#                 y_folder = f"/work/abslab/emognition_2023_challenge/data/scenario_{scenario}/train/annotations"
#                 s_folder = f"/work/abslab/emognition_2023_challenge/file_prep_30sec/scenario_1/train/cv/"

                
#                     # x_files, y_files = get_file_paths(subjects, video_numbers, x_folder, y_folder)
              
#                 train_x, train_y = get_file_paths(subjects, video_numbers, s_folder+'train/', y_folder+'train/', file_type)
#                 test_x, test_y = get_file_paths(subjects, video_numbers, s_folder+'test/', y_folder+'test/', file_type)

 
#                 fold_data = k_fold_cross_validation_time[((train_x, train_y), (test_x, test_y))]

                    
                    
            
#                 rmse_list,_,_ = TrainModel(model,fold_data,x_cols,y_columns,save_file_name,
#                                                                               verbose=False)
#                 x_names.append(x_name),rmses.append(np.mean(rmse_list))

#             pd.DataFrame({'x_col_type':x_names,
#                               'rmses':rmses}).to_csv(
#                     f'/work/abslab/emognition_2023_challenge/results/results_30s/scenario_{scenario}_{model_name}_{y_columns[0]}_{file_type}.csv')      